In [ ]:
import sys
sys.path.append('../../')

In [ ]:
# import this to use Intel optimized sklearn if available
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
from obelix_ml_pipeline.predictions_within_substrate_class_for_random_subset import predict_within_substrate_class_for_random_subset
from obelix_ml_pipeline.representation_variables import AVAILABLE_LIGAND_REPRESENTATION_TYPES, AVAILABLE_SUBSTRATE_REPRESENTATION_TYPES
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [ ]:
import os
path_to_figure_folder = os.path.join(os.getcwd(), 'figures/predictions_on_unseen_substrate')
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

# Objective 4: Predictions within a substrate class, but iteratively a smaller subset of ligands is selected so we can see the effect on the performance of the model

This case is mainly focused on regression models, since we wanted to investigate the bad performance

## Example application of pipeline

In [ ]:
# try regression with loaded representations
selected_ligand_representations = ['dft_nbd_model']
target = 'DDG'
target_threshold = None
rf_model = RandomForestRegressor(random_state=42)
scoring = 'r2'
train_splits = 5
n_jobs = int(n_cpu*0.8)  # using 0.8 of the available CPUs for parallelization
binary = False
plot_dendrograms = False
substrate_names_column = 'Substrate'
ligand_numbers_column = 'Ligand#'
selected_substrate = 'SM1'
training_size = 0.8  # 80% of data for subset substrate is used for training

subset_random_seeds = np.arange(0, 10, 1)  # 10 different random seeds for selecting a random subset of the data
print_ml_results = True
reduce_train_test_data_dimensionality = False
transformer = None
performance_threshold = 0.75  # loop of decreasing subset size and trying random seeds is stopped when performance is above this threshold

print('Training and testing regression')
print(f'Test size: {1/train_splits}')
res = predict_within_substrate_class_for_random_subset(selected_ligand_representations,
                                ligand_numbers_column, substrate_names_column, target, target_threshold, train_splits, binary,
                                selected_substrate, training_size, rf_model, scoring, print_ml_results, n_jobs, subset_random_seeds, performance_threshold, plot_dendrograms, reduce_train_test_data_dimensionality, transformer)

In [ ]:
res.fig_fi

In [ ]:
print(res.testing_performance_test)